In [ ]:
import pandas as pd
import re
from email import policy
from email.parser import BytesParser
from bs4 import BeautifulSoup
from email import message_from_string


In [ ]:
from azureml.core import Workspace,Experiment,Run

ws = Workspace.from_config(path='./config.json')


In [ ]:
data = pd.read_csv("C:/Users/bechi/Downloads/archive/emails.csv", nrows=70000)

In [ ]:
data['message'][0]


In [ ]:
import re
from email import message_from_string
from bs4 import BeautifulSoup

def clean_email(email_str):
    msg = message_from_string(email_str)
    
    # Extract basic email information
    from_address = msg.get('From')
    to_address = msg.get('To')
    subject = msg.get('Subject')
    date = msg.get('Date')
    
    body = ""
    
    if msg.is_multipart():
        for part in msg.walk():
            content_type = part.get_content_type()
            content_disposition = str(part.get("Content-Disposition"))

            if "attachment" in content_disposition:
                continue
            
            if content_type == 'text/plain':
                body += part.get_payload(decode=True).decode(errors='ignore')
            elif content_type == 'text/html':
                html_content = part.get_payload(decode=True).decode(errors='ignore')
                soup = BeautifulSoup(html_content, "html.parser")
                body += soup.get_text(separator=' ')
    else:
        body = msg.get_payload(decode=True).decode(errors='ignore')
    
    body = re.sub(r'\b(see|attached|file|attachment|https?://\S+)\b', '', body, flags=re.IGNORECASE)
    body = re.sub(r'\b[a-zA-Z]{1,2}\b', '', body)  
    body = re.sub(r'(.)\1{4,}', '', body)  
    body = re.sub(r'[a-zA-Z]{30,}', '', body)  
  
   

    
    return from_address, to_address, subject, date, body





In [ ]:
email_str = """Your raw email string here"""
from_address, to_address, subject, date, body = clean_email(email_str)
print("From:", from_address)
print("To:", to_address)
print("Subject:", subject)
print("Date:", date)
print("Body:", body)

In [ ]:
cleaned_data = pd.DataFrame([clean_email(data["message"][i]) for i in range(len(data))], 
                            columns=['from_address', 'to_address', 'subject', 'date', 'body'])

In [ ]:
cleaned_data.isnull().sum()

In [ ]:
cleaned_data = cleaned_data.dropna()
cleaned_data=cleaned_data[cleaned_data["body"]!=""]

In [ ]:
cleaned_data.head()

In [ ]:
duplicated_rows = cleaned_data.duplicated()
print(duplicated_rows.sum())
cleaned_data = cleaned_data.drop_duplicates()
len(cleaned_data)

In [ ]:
  
def extract_email(email_str):
    left = ''
    right = ''
    email = '@'
    
    if '@' in email_str:
        left, right = email_str.split('@', 1)
        
        
        
        for i in right:
            if i.isalnum() or (i == '.' and right[0] != '.'):
                email += i
            else:
                break
                
        for i in left[::-1]:
            if i.isalnum() or (i == '.' and left[-1] != '.'):
                email = i + email
            else:
                break      
    
    return email


In [ ]:
cleaned_data['from_address'] = cleaned_data['from_address'].apply(extract_email)
cleaned_data['to_address'] = cleaned_data['to_address'].apply(extract_email)

In [ ]:
print(cleaned_data[(cleaned_data['from_address'] == '@') | (cleaned_data['to_address'] == '@')])
cleaned_data = cleaned_data[(cleaned_data['from_address'] != '@') & (cleaned_data['to_address'] != '@')]
cleaned_data = cleaned_data[~(cleaned_data['from_address'].str.startswith('.') | cleaned_data['from_address'].str.endswith('.'))]
cleaned_data=cleaned_data[~(cleaned_data['to_address'].str.startswith('.') | cleaned_data['to_address'].str.endswith('.'))]
print(len(cleaned_data))
cleaned_data["date"][0].split(" ")

In [ ]:
print(len(cleaned_data[cleaned_data['date'].str[3] == ',']))
cleaned_data = cleaned_data[cleaned_data['date'].str[3] == ',']
print(len(cleaned_data))

In [ ]:
def months():
    month=[]
    a=[]
    count=0
    for date in cleaned_data['date']:
        day, subdate = date.split(',')
        if day=='':count += count
        subdate = subdate.split(' ')
        if(len(subdate)>7) |(len(subdate)<7):count += count
        if (len(subdate)==7): 
             if not subdate[2].isalpha():
               a.append(date)
             if(subdate[2]) not in month:
              month.append(subdate[2])
    print(month)  
    print(count)  
    return a

In [ ]:
a=months()
print(a)

In [ ]:
prefixes = ['re:', 'RE:', 'rE:']

filtered_data = cleaned_data[cleaned_data['subject'].apply(lambda x: any(x.startswith(prefix) for prefix in prefixes))]
print(len(filtered_data))
print(len(cleaned_data[cleaned_data['subject'].str.startswith('Re:')]))


In [ ]:
def clean_subject(subject):
    prefixes = ['re:', 'RE:', 'rE:']


    for prefix in prefixes:
        if prefix in subject:
            subject = subject.replace(prefix, 'Re:', 1)
            
    prefixes = ['Fw:']

    for prefix in prefixes:
        if subject.startswith(prefix):
            subject = subject.replace(prefix, 'FW:', 1)
            break
    return subject


In [ ]:
prefixes = ['fWD:',  'FwD:', 'fWd:', 'FWd:', 'FWD:', 'fwD:','fwd:','fW']
cleaned_data = cleaned_data[~cleaned_data['subject'].apply(lambda x: any(x.startswith(prefix) for prefix in prefixes))]
print(len(cleaned_data))

In [ ]:
cleaned_data['subject'] = cleaned_data['subject'].apply(clean_subject)
len(cleaned_data)

In [ ]:
prefixes = ['fWD:',  'FwD:', 'fWd:', 'FWd:', 'FWD:', 'fwD:','fwd:','fW']

filtered_data = cleaned_data[cleaned_data['subject'].apply(lambda x: any(prefix in x for prefix in prefixes)) ]
print(len(filtered_data['subject']))
cleaned_data = cleaned_data[~cleaned_data['subject'].apply(lambda x: any(prefix in x for prefix in prefixes))]

In [ ]:
cleaned_data.duplicated().sum()
print(len(cleaned_data))


In [ ]:
cleaned_data = cleaned_data.drop_duplicates()
print(len(cleaned_data))

In [ ]:
cleaned_data.head()

In [ ]:
def normalize_text(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

In [ ]:
cleaned_data['body'] = cleaned_data['body'].apply(normalize_text)
cleaned_data['subject'] = cleaned_data['subject'].apply(normalize_text)

In [ ]:
cleaned_data.head()

In [ ]:
cleaned_data['date'] = cleaned_data['date'].str.replace(r"\s\([A-Z]{3}\)", "")
cleaned_data["date"]= pd.to_datetime(cleaned_data['date'], utc=True)
print(cleaned_data.head())


In [ ]:
cleaned_data['year'] = cleaned_data['date'].dt.year 
cleaned_data['month'] = cleaned_data['date'].dt.month
cleaned_data['day'] = cleaned_data['date'].dt.day
cleaned_data['hour'] = cleaned_data['date'].dt.hour
cleaned_data['minute'] = cleaned_data['date'].dt.minute
cleaned_data['second'] = cleaned_data['date'].dt.second
cleaned_data['day_of_week'] = cleaned_data['date'].dt.dayofweek
cleaned_data['day_name'] = cleaned_data['date'].dt.day_name()

In [ ]:
cleaned_data.head()

In [ ]:
from nltk.corpus import stopwords
import re
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stop_words)

In [ ]:
import re

def clean_email_body(body_text):
    cleaned_body=body_text
    if isinstance(body_text, str):
        cleaned_body = re.sub(r'\b\d{1,2}:\d{2}\b', '', cleaned_body)  
        cleaned_body = re.sub(r'\bmultipart\b', '', cleaned_body, flags=re.IGNORECASE)  
        cleaned_body = re.sub(r'\s+', ' ', cleaned_body)  
        cleaned_body = re.sub(r'https?://\S+', '', cleaned_body)  
        cleaned_body = re.sub(r'\b\w{1,2}\b', '', cleaned_body)  
        cleaned_body = re.sub(r'\s+', ' ', cleaned_body)  
        cleaned_body = re.sub(r'\d+', '', cleaned_body)

        words = cleaned_body.split()
        words = [word for word in words if word not in stop_words]
        cleaned_body = ' '.join(words)

      
        return cleaned_body.strip()  
    else:
        return body_text

def clean_text(text):
            return re.sub(r'\b\w{1,2}\b', '', text).strip()  


In [ ]:
cleaned_data['body'] = cleaned_data['body'].apply(clean_email_body)

In [ ]:
cleaned_data['text'] = cleaned_data['subject'] + ' ' + cleaned_data['body']   
cleaned_data['text'] = cleaned_data['text'].apply(clean_text) 

In [ ]:
cleaned_data.head()
cleaned_data.duplicated().sum()
cleaned_data.drop_duplicates(inplace=True)
cleaned_data.duplicated().sum()
cleaned_data.isnull().sum()

In [ ]:
#import nltk
#from nltk.corpus import wordnet
#from nltk.stem import WordNetLemmatizer
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#import spacy
#nlp = spacy.load("en_core_web_sm")
# verify that words are valid English words

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

def get_valid_words(texts, threshold=0.5):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(texts)
    
    feature_names = vectorizer.get_feature_names_out()
    
    word_tfidf_sums = X.sum(axis=0).A1  
    
    word_tfidf_sum_dict = dict(zip(feature_names, word_tfidf_sums))
    
    valid_words = {word for word, score_sum in word_tfidf_sum_dict.items() if score_sum > threshold}
    
    return valid_words




In [ ]:
def clean_body_with_valid_words(texts, valid_words):
    cleaned_texts = []
    
    for body in texts:
        words = [word for word in body.split() if word in valid_words]
        cleaned_texts.append(' '.join(words))
    
    return cleaned_texts


In [ ]:
valid_words = get_valid_words(cleaned_data['text'])


In [ ]:

cleaned_data['text'] = clean_body_with_valid_words(cleaned_data['text'], valid_words)

In [ ]:

from collections import Counter
import re

def identify_abbreviations(emails):
    all_words = ' '.join(emails).split() 
    word_counter = Counter(all_words) 
    
    print(word_counter.most_common(100))

identify_abbreviations(cleaned_data['text'])

In [ ]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')

Training Model Logistic Regression


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y=label_encoder.fit_transform(cleaned_data['from_address'])

In [ ]:

vectorizer = TfidfVectorizer()
X=vectorizer.fit_transform(cleaned_data['text'])


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=0)

In [ ]:
x_train = x_train.astype(np.float32).tocsc()
x_test = x_test.astype(np.float32).tocsc()

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
model = SGDClassifier(verbose=1, n_jobs=-1)
model.fit(x_train, y_train)


In [ ]:
import joblib


joblib.dump(model, 'model.pkl')
joblib.dump(vectorizer,'vectorizer.pkl')
joblib.dump(label_encoder,'label_encoder.pkl')

In [ ]:
y_pred = model.predict(x_test)


In [ ]:
print("accuracy",model.score(x_test,y_test))
print("recall",recall_score(y_test,y_pred,average='weighted'))
print("precision",precision_score(y_test,y_pred,average='weighted'))    
print("f1_score",f1_score(y_test,y_pred,average='weighted'))


In [ ]:
print(f"Number of features in the vectorizer: {len(vectorizer.get_feature_names_out())}")
print(f"Model's expected number of features: {model.n_features_in_}")


In [ ]:
def predict_sender(query, model, vectorizer, label_encoder):
    query = clean_text(query)
    query = clean_email_body(query)
    query_vector = vectorizer.transform([query])
    query_vector = query_vector.astype(np.float32).tocsc()
    prediction = model.predict(query_vector)
    predicted_label = label_encoder.inverse_transform(prediction)
    return predicted_label[0]


In [ ]:
predict_sender('traveling business meeting takes fun trip especially prepare presentation would suggest holding business plan meetings take trip without formal business meetings would even try get honest opinions whether trip even desired necessary far business meetings think would productive try stimulate discussions across different groups working often presenter speaks others quiet waiting turn meetings might better held round table discussion format suggestion austin play golf rent ski boat jet ski flying somewhere takes much tim', model, vectorizer, label_encoder)

In [ ]:
cleaned_data[(cleaned_data['from_address']=='sally.beck@enron.com')&(cleaned_data['date']=="2001-01-11 17:59:00+00:00")]["text"]

In [ ]:
l='traveling business meeting takes fun trip especially prepare presentation would suggest holding business plan meetings take trip'.split(' ')
for index, row in cleaned_data[cleaned_data['from_address']=='phillip.allen@enron.com'].iterrows():
    if sum(word in row['text'] for word in l) >20:
        print(row['date'])

In [ ]:
cleaned_data.head()